In [3]:
# this code is writen by Alexander Firsov
# https://www.kaggle.com/alexfir/mapping-between-test-supplement-csv-and-test-csv

old_file_path = '/home/kai/data/kaggle/talkingdata/data/test_supplement.csv'
file_path = '/home/kai/data/kaggle/talkingdata/data/test.csv'
output_file_path = '/home/kai/data/kaggle/talkingdata/data/mapping.csv'


def _split(line):
    line = line.strip()
    index = line.index(',')
    last_index = line.rindex(',')
    click_id = line[:index]
    payload = line[index:]
    time = line[last_index:]
    return click_id, payload, time

def _read_same_time(lines, unprocessed_line):
    click_id, payload, group_time = _split(unprocessed_line)
    click_id_dict = {payload: [click_id]}
    while True:
        unprocessed_line = lines.readline()
        if not unprocessed_line:
            return unprocessed_line, click_id_dict, group_time
        click_id, payload, click_time = _split(unprocessed_line)
        if group_time == click_time:
            if payload in click_id_dict:
                click_id_dict[payload].append(click_id)
            else:
                click_id_dict[payload] = [click_id]
        else:
            return unprocessed_line, click_id_dict, group_time

def _find_time(lines, group_time, unprocessed_line):
    if unprocessed_line:
        click_id, payload, time = _split(unprocessed_line)
        if group_time == time:
            return unprocessed_line
    while True:
        unprocessed_line = lines.readline()
        click_id, payload, time = _split(unprocessed_line)
        if group_time == time:
            return unprocessed_line


def _save(output, test_click_id_dict, old_test_click_id_dict):
    for payload, click_ids in test_click_id_dict.items():
        old_click_ids = old_test_click_id_dict[payload]
        if len(old_click_ids) != len(click_ids):
            print('Number of ids mismatch for "{}", test ids = {}, old test ids = {}'.format(payload, click_ids,
                                                                                             old_click_ids))
        for i in range(len(click_ids)):
            output.write('{},{}\n'.format(click_ids[i], old_click_ids[i]))


with open(file_path, "r", encoding="utf-8") as test:
    with open(old_file_path, "r", encoding="utf-8") as old_test:
        with open(output_file_path, "w", encoding="utf-8") as output:
            output.write('click_id,old_click_id\n')
            test.readline()  # skip header
            old_test.readline()  # skip header
            old_test_unprocessed_line = old_test.readline()
            test_unprocessed_line = test.readline()
            while test_unprocessed_line != '':
                test_unprocessed_line, test_click_id_dict, click_time = _read_same_time(test, test_unprocessed_line)
                old_test_unprocessed_line = _find_time(old_test, click_time, old_test_unprocessed_line)
                old_test_unprocessed_line, old_test_click_id_dict, _ = _read_same_time(old_test,
                                                                                       old_test_unprocessed_line)
                _save(output, test_click_id_dict, old_test_click_id_dict)
        pass


import pandas as pd
mapping = pd.read_csv(output_file_path, dtype={'click_id': 'int32','old_click_id': 'int32'}, engine='c',
                na_filter=False,memory_map=True)



Number of ids mismatch for ",79665,12,1,17,178,2017-11-10 04:00:00", test ids = ['59'], old test ids = ['21290705', '21290936']
Number of ids mismatch for ",3859,12,1,19,178,2017-11-10 04:00:00", test ids = ['82'], old test ids = ['21290785', '21290958']
Number of ids mismatch for ",3859,8,1,19,145,2017-11-10 04:00:00", test ids = ['138'], old test ids = ['21290658', '21291016']
Number of ids mismatch for ",79665,8,1,17,145,2017-11-10 04:00:00", test ids = ['213'], old test ids = ['21290592', '21291090']
Number of ids mismatch for ",43655,8,1,15,145,2017-11-10 04:00:00", test ids = ['282'], old test ids = ['21290822', '21291157']
Number of ids mismatch for ",20264,12,1,13,178,2017-11-10 06:00:00", test ids = ['6202604'], old test ids = ['27493480', '27493896']
Number of ids mismatch for ",96692,8,1,19,145,2017-11-10 06:00:00", test ids = ['6202670'], old test ids = ['27493546', '27493810']
Number of ids mismatch for ",110078,12,1,13,178,2017-11-10 06:00:00", test ids = ['6202752'], old

In [8]:
mapping[mapping.click_id==18790099]

,click_id,old_click_id
18790099,18790099,54583887


In [9]:
mapping[mapping.old_click_id==54583887]

,click_id,old_click_id
18790099,18790099,54583887


In [10]:
mapping[mapping.old_click_id==54584310]

,click_id,old_click_id


In [4]:
mapping.shape

(18790469, 2)

In [5]:
mapping.head()

,click_id,old_click_id
0,0,21290878
1,1,21290876
2,2,21290877
3,3,21290879
4,4,21290880
